In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8/research'

In [2]:
os.chdir('../')
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    competition_name: str
    train_dataset_filename: str
    test_dataset_filename: str
    train_dataset_local_path: Path
    test_dataset_local_path: Path

In [4]:
from src.cellseg.constant import *
from src.cellseg.utils.main_utils import create_directories, read_yaml

class CofigurationMananger:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            competition_name=config.competition_name,
            train_dataset_filename=config.train_dataset_filename,
            test_dataset_filename=config.test_dataset_filename,
            train_dataset_local_path = os.path.join(
                config.root_dir,
                config.train_dataset_filename
            ),
            test_dataset_local_path = os.path.join(
                config.root_dir,
                config.test_dataset_filename
            )
        )
        
        return data_ingestion_config

In [5]:
import os
from src.cellseg.utils.main_utils import get_size, create_directories
import zipfile
import kaggle
from src.cellseg import logger

import subprocess

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_dataset(self):
        if not os.path.exists(self.config.train_dataset_local_path):
            info = subprocess.run(
                f"kaggle competitions download -c {self.config.competition_name} -f {self.config.train_dataset_filename} -p {self.config.root_dir}",
                shell=True
            )
            logger.info(f"{self.config.train_dataset_filename} downloaded with folowing info: \n{info}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.train_dataset_local_path))}")
        
        if not os.path.exists(self.config.test_dataset_local_path):
            info = subprocess.run(
                f"kaggle competitions download -c {self.config.competition_name} -f {self.config.test_dataset_filename} -p {self.config.root_dir}",
                shell=True
            )
            logger.info(f"{self.config.test_dataset_filename} downloaded with folowing info: \n{info}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.test_dataset_local_path))}")
    
    def extract_zip_files(self):
        train_unzip_path = self.config.root_dir + '/train'
        test_unzip_path = self.config.root_dir + '/test'
        
        create_directories([train_unzip_path, test_unzip_path])
        
        with zipfile.ZipFile(self.config.train_dataset_local_path, 'r') as zip_ref:
            zip_ref.extractall(train_unzip_path)
        
        os.remove(self.config.train_dataset_local_path)
        
        with zipfile.ZipFile(self.config.test_dataset_local_path, 'r') as zip_ref:
            zip_ref.extractall(test_unzip_path)
        
        os.remove(self.config.test_dataset_local_path)

In [6]:
try:
    config = CofigurationMananger()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_dataset()
    data_ingestion.extract_zip_files()

except Exception as e:
    raise e

[2024-10-01 22:01:32,696: INFO: main_utils: created directory at: artifacts]
[2024-10-01 22:01:32,698: INFO: main_utils: created directory at: artifacts/data_ingestion]


 91%|█████████ | 72.0M/79.1M [00:03<00:00, 34.3MB/s]


[2024-10-01 22:01:37,373: INFO: 3321230123: stage1_train.zip downloaded with folowing info: 
CompletedProcess(args='kaggle competitions download -c data-science-bowl-2018 -f stage1_train.zip -p artifacts/data_ingestion', returncode=0)]


100%|██████████| 79.1M/79.1M [00:03<00:00, 24.7MB/s]


 22%|██▏       | 2.00M/9.10M [00:00<00:02, 2.58MB/s]


[2024-10-01 22:01:40,336: INFO: 3321230123: stage1_test.zip downloaded with folowing info: 
CompletedProcess(args='kaggle competitions download -c data-science-bowl-2018 -f stage1_test.zip -p artifacts/data_ingestion', returncode=0)]
[2024-10-01 22:01:40,338: INFO: main_utils: created directory at: artifacts/data_ingestion/train]
[2024-10-01 22:01:40,338: INFO: main_utils: created directory at: artifacts/data_ingestion/test]


100%|██████████| 9.10M/9.10M [00:01<00:00, 7.85MB/s]
